In [1]:
!pip install textblob

In [2]:
pip install tweet-preprocessor

Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
sys.path.append('/usr/local/bin/python2.7')


import numpy as np
import os, sys, getopt, pickle, csv, sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix, make_scorer, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
# from sklearn.grid_search import GridSearchCV
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from textblob import TextBlob
import random
import matplotlib.pyplot as plt
from sklearn import metrics
from collections import Counter
import argparse
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score    
import preprocessor as p

In [4]:
models = [ 'svm', 'naive', 'lr', 'random_forest']
NO_OF_FOLDS =13
MODEL_TYPE = "all"
HASH_REMOVE = None

In [5]:
def load_data(filename):
    data = pickle.load(open(filename, 'rb'))
    x_text = []
    labels = []
    for i in range(len(data)):
        if(HASH_REMOVE):
            x_text.append(p.tokenize((data[i]['text']).encode('utf-8')))
        else:
            x_text.append(data[i]['text'])
        labels.append(data[i]['label'])
    return x_text,labels

def get_filename(dataset):
    global N_CLASS, HASH_REMOVE
    if(dataset=="twitter"):
        filename = "twitter_data.pkl"
        N_CLASS = 3
        HASH_REMOVE = False
    elif(dataset=="formspring"):
        N_CLASS = 2
        filename = "formspring_data.pkl"
        HASH_REMOVE = False
    elif(dataset=="wiki"):
        N_CLASS = 2
        filename = "wiki_data.pkl"
        HASH_REMOVE = False
    return filename

In [6]:
def get_scores(y_true, y_pred):
#     if(data=="wiki"):
#         auc = roc_auc_score(y_true,y_pred)
#         print('Test ROC AUC: %.3f' %auc)
#     print(":: Confusion Matrix")
#     print(confusion_matrix(y_true, y_pred))
#     print(":: Classification Report")
#     print(classification_report(y_true, y_pred))
    return np.array([ 
            precision_score(y_true, y_pred, average=None), 
            recall_score(y_true, y_pred,  average=None),
            f1_score(y_true, y_pred, average=None)])
    
def print_scores(scores):
    for i in range(N_CLASS):
        if(i!=0):
            print ("Precision Class %d (avg): %0.3f (+/- %0.3f)" % (i,scores[:, i].mean(), scores[:, i].std() * 2))
            print ("Recall Class %d (avg): %0.3f (+/- %0.3f)" % (i,scores[:,  N_CLASS+i].mean(), scores[:,N_CLASS+i].std() * 2))
            print ("F1_score Class %d (avg): %0.3f (+/- %0.3f)" % (i,scores[:, N_CLASS*2+i].mean(), scores[:,  N_CLASS*2+i].std() * 2))

In [7]:
def classification_model(X, Y, model_type):
    X, Y = shuffle(X, Y, random_state=42)
    print ("Model Type:", model_type)
    kf = KFold(n_splits=NO_OF_FOLDS)
    scores = []
    for train_index, test_index in kf.split(X):
        Y = np.asarray(Y)
        model = get_model(model_type)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        curr_scores = get_scores(y_test, y_pred)
        scores.append(np.hstack(curr_scores))
    print_scores(np.array(scores))

In [8]:
def get_model(m_type):
    if m_type == 'lr':
        logreg = LogisticRegression(class_weight="balanced")
    elif m_type == 'naive':
        logreg =  MultinomialNB()
    elif m_type == "random_forest":
        logreg = RandomForestClassifier(n_estimators=100, n_jobs=-1)
    elif m_type == "svm":
        logreg = LinearSVC(class_weight="balanced")
    else:
        print ("ERROR: Please specify a correst model")
        return None
    return logreg

In [ ]:
def train(x_text, labels, MODEL_TYPE):
    
    if(WORD):
        print("Using word based features")
        bow_transformer = CountVectorizer(analyzer="word",max_features = 10000,stop_words='english').fit(x_text)
        comments_bow = bow_transformer.transform(x_text)
        tfidf_transformer = TfidfTransformer(norm = 'l2').fit(comments_bow)
        comments_tfidf = tfidf_transformer.transform(comments_bow)
        features = comments_tfidf
    else: 
        print("Using char n-grams based features")
        bow_transformer = CountVectorizer(max_features = 10000, ngram_range = (1,2)).fit(x_text)
        comments_bow = bow_transformer.transform(x_text)
        tfidf_transformer = TfidfTransformer(norm = 'l2').fit(comments_bow)
        comments_tfidf = tfidf_transformer.transform(comments_bow)
        features = comments_tfidf
    
    if(data == "twitter"):
        dict1 = {'racism':0,'sexism':1,'none':2}
        labels = np.array([dict1[b] for b in labels])
    
    from collections import Counter
    print(Counter(labels))
    
    if(MODEL_TYPE != "all"):
        classification_model(features, labels, MODEL_TYPE)
    else:
        for model_type in models:
            classification_model(features, labels, model_type)

In [10]:
data = "twitter"
WORD = True
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using word based features
Counter({2: 11036, 1: 3117, 0: 1937})
Model Type: svm


D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change

Precision Class 1 (avg): 0.803 (+/- 0.026)
Recall Class 1 (avg): 0.749 (+/- 0.054)
F1_score Class 1 (avg): 0.775 (+/- 0.033)
Precision Class 2 (avg): 0.894 (+/- 0.025)
Recall Class 2 (avg): 0.900 (+/- 0.016)
F1_score Class 2 (avg): 0.897 (+/- 0.009)
Model Type: naive
Precision Class 1 (avg): 0.907 (+/- 0.024)
Recall Class 1 (avg): 0.479 (+/- 0.064)
F1_score Class 1 (avg): 0.627 (+/- 0.057)
Precision Class 2 (avg): 0.809 (+/- 0.028)
Recall Class 2 (avg): 0.963 (+/- 0.007)
F1_score Class 2 (avg): 0.879 (+/- 0.015)
Model Type: lr


D:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result

Precision Class 1 (avg): 0.757 (+/- 0.034)
Recall Class 1 (avg): 0.789 (+/- 0.063)
F1_score Class 1 (avg): 0.773 (+/- 0.037)
Precision Class 2 (avg): 0.914 (+/- 0.026)
Recall Class 2 (avg): 0.861 (+/- 0.019)
F1_score Class 2 (avg): 0.886 (+/- 0.012)
Model Type: random_forest
Precision Class 1 (avg): 0.875 (+/- 0.048)
Recall Class 1 (avg): 0.649 (+/- 0.086)
F1_score Class 1 (avg): 0.745 (+/- 0.071)
Precision Class 2 (avg): 0.868 (+/- 0.032)
Recall Class 2 (avg): 0.937 (+/- 0.014)
F1_score Class 2 (avg): 0.901 (+/- 0.020)


In [11]:
data = "formspring"
WORD =  False
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using char n-grams based features
Counter({0: 11997, 1: 776})
Model Type: svm


D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change

Precision Class 1 (avg): 0.458 (+/- 0.124)
Recall Class 1 (avg): 0.513 (+/- 0.143)
F1_score Class 1 (avg): 0.483 (+/- 0.128)
Model Type: naive
Precision Class 1 (avg): 0.692 (+/- 0.923)
Recall Class 1 (avg): 0.015 (+/- 0.025)
F1_score Class 1 (avg): 0.029 (+/- 0.049)
Model Type: lr


D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision Class 1 (avg): 0.408 (+/- 0.091)
Recall Class 1 (avg): 0.623 (+/- 0.122)
F1_score Class 1 (avg): 0.493 (+/- 0.097)
Model Type: random_forest
Precision Class 1 (avg): 0.755 (+/- 0.246)
Recall Class 1 (avg): 0.159 (+/- 0.091)
F1_score Class 1 (avg): 0.261 (+/- 0.129)


In [12]:
data = "formspring"
WORD = True
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using word based features
Counter({0: 11997, 1: 776})
Model Type: svm


D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change

Precision Class 1 (avg): 0.417 (+/- 0.104)
Recall Class 1 (avg): 0.531 (+/- 0.153)
F1_score Class 1 (avg): 0.465 (+/- 0.108)
Model Type: naive
Precision Class 1 (avg): 0.513 (+/- 0.965)
Recall Class 1 (avg): 0.016 (+/- 0.035)
F1_score Class 1 (avg): 0.031 (+/- 0.066)
Model Type: lr
Precision Class 1 (avg): 0.400 (+/- 0.085)
Recall Class 1 (avg): 0.611 (+/- 0.153)
F1_score Class 1 (avg): 0.482 (+/- 0.095)
Model Type: random_forest
Precision Class 1 (avg): 0.738 (+/- 0.303)
Recall Class 1 (avg): 0.182 (+/- 0.115)
F1_score Class 1 (avg): 0.289 (+/- 0.166)


In [13]:
data = "twitter"
WORD = False
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using char n-grams based features
Counter({2: 11036, 1: 3117, 0: 1937})
Model Type: svm


D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change

Precision Class 1 (avg): 0.784 (+/- 0.047)
Recall Class 1 (avg): 0.732 (+/- 0.070)
F1_score Class 1 (avg): 0.756 (+/- 0.049)
Precision Class 2 (avg): 0.890 (+/- 0.025)
Recall Class 2 (avg): 0.896 (+/- 0.017)
F1_score Class 2 (avg): 0.893 (+/- 0.013)
Model Type: naive
Precision Class 1 (avg): 0.916 (+/- 0.047)
Recall Class 1 (avg): 0.459 (+/- 0.070)
F1_score Class 1 (avg): 0.610 (+/- 0.066)
Precision Class 2 (avg): 0.806 (+/- 0.026)
Recall Class 2 (avg): 0.963 (+/- 0.009)
F1_score Class 2 (avg): 0.877 (+/- 0.015)
Model Type: lr


D:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result

Precision Class 1 (avg): 0.741 (+/- 0.041)
Recall Class 1 (avg): 0.774 (+/- 0.076)
F1_score Class 1 (avg): 0.757 (+/- 0.046)
Precision Class 2 (avg): 0.910 (+/- 0.029)
Recall Class 2 (avg): 0.853 (+/- 0.018)
F1_score Class 2 (avg): 0.880 (+/- 0.013)
Model Type: random_forest
Precision Class 1 (avg): 0.897 (+/- 0.060)
Recall Class 1 (avg): 0.569 (+/- 0.087)
F1_score Class 1 (avg): 0.695 (+/- 0.078)
Precision Class 2 (avg): 0.844 (+/- 0.031)
Recall Class 2 (avg): 0.950 (+/- 0.016)
F1_score Class 2 (avg): 0.894 (+/- 0.021)


In [14]:
data = "wiki"
WORD = False
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using char n-grams based features
Counter({0: 102274, 1: 13590})
Model Type: svm


D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change

Precision Class 1 (avg): 0.593 (+/- 0.027)
Recall Class 1 (avg): 0.824 (+/- 0.020)
F1_score Class 1 (avg): 0.690 (+/- 0.018)
Model Type: naive
Precision Class 1 (avg): 0.839 (+/- 0.025)
Recall Class 1 (avg): 0.555 (+/- 0.025)
F1_score Class 1 (avg): 0.668 (+/- 0.021)
Model Type: lr


D:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result

Precision Class 1 (avg): 0.603 (+/- 0.029)
Recall Class 1 (avg): 0.846 (+/- 0.025)
F1_score Class 1 (avg): 0.704 (+/- 0.023)
Model Type: random_forest
Precision Class 1 (avg): 0.888 (+/- 0.014)
Recall Class 1 (avg): 0.550 (+/- 0.023)
F1_score Class 1 (avg): 0.679 (+/- 0.018)


In [15]:
data = "wiki"
WORD = True
x_text, labels = load_data(get_filename(data)) 
print ("Data loaded!")
train(x_text, labels, MODEL_TYPE)

Data loaded!
Using word based features
Counter({0: 102274, 1: 13590})
Model Type: svm


D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\Anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change

Precision Class 1 (avg): 0.592 (+/- 0.027)
Recall Class 1 (avg): 0.819 (+/- 0.029)
F1_score Class 1 (avg): 0.687 (+/- 0.023)
Model Type: naive
Precision Class 1 (avg): 0.897 (+/- 0.027)
Recall Class 1 (avg): 0.522 (+/- 0.030)
F1_score Class 1 (avg): 0.660 (+/- 0.028)
Model Type: lr


D:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Precision Class 1 (avg): 0.619 (+/- 0.028)
Recall Class 1 (avg): 0.834 (+/- 0.025)
F1_score Class 1 (avg): 0.711 (+/- 0.022)
Model Type: random_forest
Precision Class 1 (avg): 0.811 (+/- 0.026)
Recall Class 1 (avg): 0.664 (+/- 0.027)
F1_score Class 1 (avg): 0.730 (+/- 0.021)
